## Recap курса "Введение в машинное обучение"

## Семинар #10

### Екатерина Кондратьева

ekaterina.kondrateva@skoltech.ru

  Программа курса:
  
   1. Обучение с учителем: Линейная и логистическая регрессия. Ядра.
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/02_lr
    
 
    
   2. Обучение с учителем: Регуляризация в линейных моделях. Метод Ближайших Соседей (KNN)
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/blob/master/03_knn
    

    
   3. Обучение с учителем: Метод опорных векторов (SVM) для задач классификации и регрессии. Kernel SVM
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/04_svc
     

    
   4. Обучение с учителем: Деревья решений (Decision Trees). Случайный лес (Random Forest).
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/05_rfc
    

    
   5. Оценка качества алгоритмов машинного обучения. Кросс-валидация. Поиск аномалий и артефактов в выборке. 
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/06_model_validaton
    

    
   6. Обучение без учителя: кластеризация. Снижение размерности данных PCA.
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/07_clustering_pca
    

    
   7. Отбор и генерация признаков (Feature Engineering). Поиск и оптимизация модели (Grid Search).
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/08_feature_engineering
    

    
   8. Стекинг, вотинг. Градиентный бустинг. Пакеты XGBoost/Catboost/LightGBM  
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/09_gradient_boosting
    
    

### Построим пайплан для датасета HousePricing:

Данные - https://www.kaggle.com/lider123/housepricing

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [2]:
train_data = pd.read_csv("housepricing/train.csv")
test_data = pd.read_csv("housepricing/test.csv")

In [3]:
full_data = pd.concat([train_data,test_data])

C:\01Dev\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [4]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 81 columns):
1stFlrSF         2919 non-null int64
2ndFlrSF         2919 non-null int64
3SsnPorch        2919 non-null int64
Alley            198 non-null object
BedroomAbvGr     2919 non-null int64
BldgType         2919 non-null object
BsmtCond         2837 non-null object
BsmtExposure     2837 non-null object
BsmtFinSF1       2918 non-null float64
BsmtFinSF2       2918 non-null float64
BsmtFinType1     2840 non-null object
BsmtFinType2     2839 non-null object
BsmtFullBath     2917 non-null float64
BsmtHalfBath     2917 non-null float64
BsmtQual         2838 non-null object
BsmtUnfSF        2918 non-null float64
CentralAir       2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
Electrical       2918 non-null object
EnclosedPorch    2919 non-null int64
ExterCond        2919 non-null object
ExterQual        2919 non-null object
Exterior1st      291

In [5]:
categorial_features = [prizn for prizn in full_data.columns if full_data[prizn].dtype.name == 'object']
numeric_features = [prizn for prizn in full_data.columns if full_data[prizn].dtype.name != 'object']
print(categorial_features, numeric_features, sep = '\n')

['Alley', 'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2', 'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd', 'Fence', 'FireplaceQu', 'Foundation', 'Functional', 'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC', 'HouseStyle', 'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig', 'LotShape', 'MSZoning', 'MasVnrType', 'MiscFeature', 'Neighborhood', 'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle', 'SaleCondition', 'SaleType', 'Street', 'Utilities']
['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars', 'GarageYrBlt', 'GrLivArea', 'HalfBath', 'Id', 'KitchenAbvGr', 'LotArea', 'LotFrontage', 'LowQualFinSF', 'MSSubClass', 'MasVnrArea', 'MiscVal', 'MoSold', 'OpenPorchSF', 'OverallCond', 'OverallQual', 'PoolArea', 'SalePrice',

In [6]:
# заполним пропуски числовых данных применяя свои знания предметной области
full_data["SaleType"].fillna(full_data["SaleType"].mode()[0], inplace=True)
full_data["Electrical"].fillna(full_data["Electrical"].mode()[0], inplace=True)
full_data["MSZoning"].fillna(full_data["MSZoning"].mode()[0], inplace=True)
full_data["GarageArea"].fillna(full_data["GarageArea"].median(), inplace=True)
full_data["CentralAir"] = full_data["CentralAir"].map({"Y": 1, "N": 0})
full_data["LotFrontage"].fillna(full_data["LotFrontage"].median(), inplace=True)
full_data["Alley"].fillna("None", inplace=True)
full_data["MasVnrType"].fillna("None", inplace=True)
full_data["MasVnrArea"].fillna(0, inplace=True)

In [7]:
#заполним пропуски числовых данных подряд медианой
for i in numeric_features:
    full_data[i].fillna(full_data[i].median(), inplace=True)

In [8]:
# кодируем переменные типа объекта
full_data = pd.get_dummies(full_data, columns=categorial_features)

X_train, X_test = full_data.iloc[:len(train_data), :], full_data.iloc[len(train_data):, :]

X_train.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave,Utilities_AllPub,Utilities_NoSeWa
0,856,854,0,3,706.0,0.0,1.0,0.0,150.0,0,...,0,0,0,0,0,1,0,1,1,0
1,1262,0,0,3,978.0,0.0,0.0,1.0,284.0,0,...,0,0,0,0,0,1,0,1,1,0
2,920,866,0,3,486.0,0.0,1.0,0.0,434.0,0,...,0,0,0,0,0,1,0,1,1,0
3,961,756,0,3,216.0,0.0,1.0,0.0,540.0,272,...,0,0,0,0,0,1,0,1,1,0
4,1145,1053,0,4,655.0,0.0,1.0,0.0,490.0,0,...,0,0,0,0,0,1,0,1,1,0


In [9]:
X_train.shape

(1460, 291)

#### Снижение размерности?
#### Какая задача - регрессия или классификация?

In [10]:
# построим самый простой регрессор
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, StratifiedKFold

clf = LinearRegression()

#### Регуляризация?
#### Оценка модели? 

In [11]:
y_train = train_data["SalePrice"]
X_train.drop("SalePrice",axis=1, inplace=True)

C:\01Dev\anaconda\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
y_train.shape

(1460,)

In [13]:
X = X_train
y = y_train
skf = StratifiedKFold(n_splits=5, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    scores.append(clf.score(X_test, y_test))

C:\01Dev\anaconda\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


-485532234.90560126
-1326514097.2557805
-24411679.341038484
-3502214323.4653296
-921707527593.5194


In [14]:
from sklearn.preprocessing import StandardScaler

sclr= StandardScaler()
X_cols= X_train.columns
X = pd.DataFrame(sclr.fit_transform(X_train), columns = X_cols)


y = y_train
skf = StratifiedKFold(n_splits=5, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    scores.append(clf.score(X_test, y_test))

C:\01Dev\anaconda\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\01Dev\anaconda\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\01Dev\anaconda\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


-1.8126751392592524e+25
-7.483752725566383e+24
-5.012779502932759e+26
-6.538233444074639e+23
-2.559422889835046e+26


In [15]:
cross_val_score(lrr, X_train, y_train, cv=5)

NameError: name 'lrr' is not defined

In [16]:
X_train_short= X_train[['Id', 'MSSubClass', 'LotArea', 'OverallQual',\
      'OverallCond', 'YearBuilt', 'YearRemodAdd',\
      'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',\
     'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',\
     'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',\
     'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',\
     'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',\
     'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']]

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

params = {
    "n_estimators": [10, 50],
    "max_features": ["auto", "sqrt"],
    "max_depth": [5,10]
}
gs = GridSearchCV(estimator=RandomForestRegressor(), param_grid=params, cv=5, scoring="neg_mean_squared_error")
gs.fit(X_train, y_train)

gs.best_params_

C:\01Dev\anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'max_depth': 10, 'max_features': 'auto', 'n_estimators': 50}

In [18]:
model = RandomForestRegressor(**gs.best_params_)
cross_val_score(model, X_train, y_train, cv=5)

array([0.84802855, 0.8999728 , 0.80345994, 0.87841662, 0.86847025])

In [19]:
X = X_train
y = y_train
skf = StratifiedKFold(n_splits=5, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))
    scores.append(model.score(X_test, y_test))

C:\01Dev\anaconda\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


0.7332598493917576
0.8588661264979394
0.8076380430624518
0.7750695708465714
0.6511126803834564


In [20]:
X = X_train
y = y_train
skf = StratifiedKFold(n_splits=5, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    scores.append(clf.score(X_test, y_test))

C:\01Dev\anaconda\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


-4.193652338726335e+25
-1.0929071673599224e+25
-2.054326129964796e+25
-1.5386759094446282e+26
0.5245159854234116


## Titanic dataset 

In [ ]:
train_df = pd.read_csv('titanic/train.csv')
test_df = pd.read_csv('titanic/test.csv')

In [ ]:
train_df["Male"] = train_df.Sex.map({"male": 1, "female": 0}).astype(int) # альтернатива OneHotEncoding
test_df["Male"] = test_df.Sex.map({"male": 1, "female": 0}).astype(int)

train_df = train_df.drop(["PassengerId", "Ticket", "Cabin", "Name", "Sex", "SibSp", "Parch", "Embarked", "Fare", 'Age'], axis=1)
test_df = test_df.drop(["Ticket", "Cabin", "Name", "Sex", "SibSp", "Parch", "Embarked", "Fare",'Age'], axis=1)

X = train_df.drop("Survived", axis=1).fillna(np.nan)
y = train_df.Survived
X_test = test_df.drop("PassengerId", axis=1).fillna(np.nan)


X.tail()

In [ ]:
from sklearn.linear_model import LogisticRegression
clf= LogisticRegression()

In [ ]:
skf = StratifiedKFold(n_splits=10, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    scores.append(clf.score(X_test, y_test))

In [ ]:
clf = Pipeline([('selectk', SelectKBest(k=6)), 
                     ('scaler', StandardScaler()),
                ('decision_tree',  RandomForestClassifier(n_estimators=25, max_depth=3))])

cross_val_score(estimator=clf, X=X_train, y=y_train, cv=5).mean()

### Метрики: оценки качества

### Практикум по *.Git системам:

!['git'](https://i1.wp.com/lanziani.com/slides/gitflow/images/gitflow_1.png?zoom=2)

1. Установка: https://git-scm.com/downloads
2. GUI: https://desktop.github.com/
3. Туториал: https://learngitbranching.js.org/